In [21]:
import pandas as pd
import pickle as pk

In [22]:
with open('../src/depara/trends/categoria.pkl', 'rb') as arq:
    df_categorias = pk.load(arq)
    df_categorias['ID'] = df_categorias['ID'].astype('int32')
    df_categorias['NOME_CATEGORIA'] = df_categorias['NOME_CATEGORIA'].astype('string')
df_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              31 non-null     int32 
 1   NOME_CATEGORIA  31 non-null     string
dtypes: int32(1), string(1)
memory usage: 500.0 bytes


In [23]:
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet')

In [24]:
base_trends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_extracao          4400 non-null   string 
 1   DATA_PUBLICACAO        4400 non-null   string 
 2   ID_CANAL               4400 non-null   string 
 3   NM_CANAL               4400 non-null   string 
 4   ID_CATEGORIA           4400 non-null   int32  
 5   ID_VIDEO               4400 non-null   string 
 6   TITULO_VIDEO           4400 non-null   string 
 7   DESCRICAO              4400 non-null   string 
 8   TAGS                   3661 non-null   string 
 9   DURACAO_VIDEOS         4400 non-null   object 
 10  TOTAL_VISUALIZACOES    4400 non-null   int32  
 11  TOTAL_LIKES            4400 non-null   int32  
 12  TOTAL_FAVORITOS        4400 non-null   int32  
 13  TOTAL_COMENTARIOS      4400 non-null   int32  
 14  data_hora_extracao     4400 non-null   string 
 15  DURA

# Métricas Trends

- Categorias Populares por data de extração top 10

In [56]:
#  'TOTAL_VISUALIZACOES', 'TOTAL_LIKES', 'TOTAL_COMENTARIOS'
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'ID_VIDEO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' data_extracao == "2024-01-18" and INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='ID_VIDEO')

,data_extracao,ID_CATEGORIA,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_VIDEO,TOTAL_VISUALIZACOES
2507,2024-01-18,24,Noite,3,-7a68_XfpG4,679320
1551,2024-01-18,17,Noite,3,-A7X9LMN0qU,114810
41,2024-01-18,17,Noite,3,-JzAKMfuWy4,223287
733,2024-01-18,24,Noite,3,-U5oLyhfq0A,172139
3672,2024-01-18,24,Noite,3,-cGZ4mFlAFY,325350
...,...,...,...,...,...,...
1096,2024-01-18,20,Noite,3,yJzvSZ7EKN8,1451661
987,2024-01-18,20,Noite,3,yN1u45QKPVw,152720
4077,2024-01-18,17,Noite,3,yQPIQ97nYG4,278899
2590,2024-01-18,28,Noite,3,zKmNviriS50,65072


In [57]:
base_trends.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 41 to 4357
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   data_extracao          200 non-null    string
 1   ID_CATEGORIA           200 non-null    int32 
 2   TURNO_EXTRACAO         200 non-null    string
 3   INDICE_TURNO_EXTRACAO  200 non-null    string
 4   ID_VIDEO               200 non-null    string
 5   TOTAL_VISUALIZACOES    200 non-null    int32 
dtypes: int32(2), string(4)
memory usage: 9.4 KB


In [59]:
base_trends = base_trends.groupby(['data_extracao', 'ID_CATEGORIA']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')

    ).reset_index()

base_trends

,data_extracao,ID_CATEGORIA,TOTAL_MAX
0,2024-01-18,1,823820
1,2024-01-18,2,161708
2,2024-01-18,10,81801902
3,2024-01-18,17,30452063
4,2024-01-18,19,1074972
5,2024-01-18,20,124769848
6,2024-01-18,22,12286382
7,2024-01-18,23,2277480
8,2024-01-18,24,125759485
9,2024-01-18,25,5939219


In [60]:
base_trends_completa = pd.merge(
    right=base_trends,
    left=df_categorias,
    how='inner',
    left_on='ID',
    right_on='ID_CATEGORIA'
).sort_values(by='TOTAL_MAX', ascending=False)
base_trends_completa

,ID,NOME_CATEGORIA,data_extracao,ID_CATEGORIA,TOTAL_MAX
8,24,Entertainment,2024-01-18,24,125759485
5,20,Gaming,2024-01-18,20,124769848
2,10,Music,2024-01-18,10,81801902
3,17,Sports,2024-01-18,17,30452063
6,22,People & Blogs,2024-01-18,22,12286382
11,28,Science & Technology,2024-01-18,28,7185043
9,25,News & Politics,2024-01-18,25,5939219
7,23,Comedy,2024-01-18,23,2277480
4,19,Travel & Events,2024-01-18,19,1074972
0,1,Film & Animation,2024-01-18,1,823820


In [44]:
base_trends_completa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 99 to 73
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              209 non-null    int32 
 1   NOME_CATEGORIA  209 non-null    string
 2   data_extracao   209 non-null    string
 3   ID_CATEGORIA    209 non-null    int32 
 4   ID_VIDEO        209 non-null    string
 5   TOTAL_MAX       209 non-null    int32 
dtypes: int32(3), string(3)
memory usage: 9.0 KB


- Canais populares (likes, comentarios e visualizações) Por categoria

In [71]:
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' ID_CATEGORIA == 20 and data_extracao == "2024-01-19"  and  INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='ID_CANAL')

,data_extracao,ID_CATEGORIA,ID_CANAL,NM_CANAL,ID_VIDEO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES
1727,2024-01-19,20,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,ZHhqwBwmRkI,Noite,3,104889364
2459,2024-01-19,20,UC6P6SvP2arhEdJ9Pg8qwP2A,Athos,RVoUKeKn7q0,Noite,3,705586
2777,2024-01-19,20,UC8rAKIddyMROk00d6Kebi0g,KillerProductionsful,34ei8y1vjOU,Noite,3,557654
2110,2024-01-19,20,UC9Qu6o04oTf-b2acahQVk1g,Kass e KR,WZMFR2HHPto,Noite,3,857416
920,2024-01-19,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,RApm1yPoE-0,Noite,3,157753
935,2024-01-19,20,UCAu_v9w-BXFtg1NAnB-JgVg,Apuh,g7bRCHLCs1g,Noite,3,156645
2423,2024-01-19,20,UCBiRuk3NRX_SxCsp3l3cLAw,Xou,wq0TJjlggrM,Noite,3,72519
2202,2024-01-19,20,UCEfGV5hx2VrXl4jOUnG0MRQ,Geleia,xmnrb2Q6D44,Noite,3,803370
806,2024-01-19,20,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,yJzvSZ7EKN8,Noite,3,1676719
1308,2024-01-19,20,UCIXguhHCl8eDTkXpEuiGPUA,Jazzghost,kVzwBImBoGU,Noite,3,1325611


In [72]:
base_trends.groupby(['data_extracao', 'ID_CANAL']) \
    .agg(
        TOTAL_MAX=(metrica, 'sum')
    ).reset_index().sort_values(by='TOTAL_MAX', ascending=False)

,data_extracao,ID_CANAL,TOTAL_MAX
0,2024-01-19,UC2t5bjwHdUX4vM2g8TRDq5g,104889364
7,2024-01-19,UCIXguhHCl8eDTkXpEuiGPUA,4517914
29,2024-01-19,UCrcBZDofBltcgGxphNia2tw,2975865
17,2024-01-19,UCWsDFcIhY2DBi3GB5uykGXA,1295946
15,2024-01-19,UCVse66mti9GN2SWF2eh_SCg,1226690
23,2024-01-19,UCiS882YPwZt1NfaM0gR0D9Q,1177554
24,2024-01-19,UCjBp_7RuDBUYbd1LegWEJ8g,1071090
3,2024-01-19,UC9Qu6o04oTf-b2acahQVk1g,857416
6,2024-01-19,UCEfGV5hx2VrXl4jOUnG0MRQ,803370
21,2024-01-19,UCeoaBlUFa1wvkXnkYgsAY3A,708432


- Desempenho da categoria por dia

In [ ]:
metrica = 'TOTAL_VISUALIZACOES'

colunas = ['data_extracao', 'ID_CATEGORIA' , 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO', metrica]
base_trends = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_trends.parquet', columns=colunas)
base_trends = base_trends.query(' ID_CATEGORIA == 20 and data_extracao == "2024-01-19"  and  INDICE_TURNO_EXTRACAO == "3" ')
base_trends.sort_values(by='ID_CANAL')

- Top 10 vídeo em alta por categoria

- Canais populares (likes, comentarios e visualizações) 